<a href="https://colab.research.google.com/github/MelchiorReihlenBorgers/EstimatingTheWeightOfCows/blob/master/BoundaryBoxEstimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import scale
from sklearn.metrics import plot_precision_recall_curve

import matplotlib.pyplot as plt

# Load Data

In [ ]:
os.chdir("/content/drive/My Drive/Master/Thesis")

classification_data = pd.read_csv("classifications_data.csv", header = 0)

classification_data.head()

,Unnamed: 0,Mean_Depth,Mean_Saliency,Color_Contrast,Edge_Density,SD_Depth,SD_Saliency,Depth_Contrast,Binary_Saliency,label
0,0,0.789158,0.173722,575036.848217,17.852036,0.023635,0.155314,66584442,0.401331,1
1,1,0.786952,0.174417,623808.873008,13.995223,0.023951,0.162438,84934794,0.401107,1
2,2,0.792735,0.246356,238612.785629,30.235564,0.023659,0.175280,103448285,0.483905,1
3,3,0.789762,0.209529,484801.779998,20.077211,0.024169,0.173342,89789188,0.437259,1
4,4,0.786636,0.158779,827020.097792,15.696170,0.025097,0.152270,114788330,0.387066,1


In [ ]:
col_names = ["Mean_Depth", "Mean_Saliency", "Color_Contrast", "Edge_Density", "SD_Depth", "SD_Saliency",
             "Depth_Contrast", "Binary_Saliency"]

X = classification_data.loc[:,col_names]

In [ ]:
X.loc[:,["Color_Contrast", "Depth_Contrast"]] = X.loc[:,["Color_Contrast", "Depth_Contrast"]]/1000
X = scale(X)

y = classification_data.loc[:,"label"]

# Train Test Split

With Depth

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 0)

No Depth

In [ ]:
X_train_ND, X_test_ND = np.delete(arr = X_train, obj = [0,4,6], axis = 1), np.delete(arr = X_test, obj = [0,4,6], axis = 1)

# Define Classifiers

In [ ]:
nb = GaussianNB()
knn = KNeighborsClassifier()
svc = SVC(probability= True)

In [ ]:
methods = {"Naive Bayes":nb,
           "KNN": knn,
           "SVM": svc}

# Make Predictions

In [ ]:
fit = nb.fit(X_train, y_train)

predict_nb = fit.predict_proba(X_test)

precision_nb, recall_nb, thresholds_nb = precision_recall_curve(y_true = y_test, probas_pred = predict_nb[:,0])

In [ ]:
fit = knn.fit(X_train, y_train)

predict_knn = fit.predict_proba(X_test)

precision_knn, recall_knn, thresholds_knn = precision_recall_curve(y_true = y_test, probas_pred = predict_knn[:,0])

In [1]:
fit = svc.fit(X_train, y_train)

predict_svm = fit.predict_proba(X_test)

precision_svm, recall_svm, thresholds_svm = precision_recall_curve(y_true = y_test, probas_pred = predict_svm[:,0])

NameError: ignored

# Save mAP 

Including Depth

In [ ]:
precisions = []


for method_name, method in methods.items():

    # Fit model
    fit = method.fit(X_train, y_train)

    # Predict in-sample
    prediction_in_sample = fit.predict(X_train)
    maP_in_sample = average_precision_score(y_true = y_train, y_score = prediction_in_sample)

    # Predict out-of-sample
    prediction_out_of_sample = fit.predict(X_test)
    maP_out_of_sample = average_precision_score(y_true=y_test, y_score=prediction_out_of_sample)

    precision = (maP_in_sample, maP_out_of_sample)

    precisions.append(precision)

precisions = pd.DataFrame(precisions, columns = ["Train", "Test"], index = [method for method, x in methods.items()])

Excluding Depth

In [ ]:
precisions_ND = []

for method_name, method in methods.items():
    # Fit model
    fit = method.fit(X_train_ND, y_train)

    # Predict in-sample
    prediction_in_sample_ND = fit.predict(X_train_ND)
    maP_in_sample_ND = average_precision_score(y_true = y_train, y_score = prediction_in_sample_ND)

    # Predict out-of-sample
    prediction_out_of_sample_ND = fit.predict(X_test_ND)
    maP_out_of_sample_ND = average_precision_score(y_true=y_test, y_score=prediction_out_of_sample_ND)

    accuracy = (accuracy_in_sample_ND, accuracy_out_of_sample_ND)
    precision = (maP_in_sample_ND, maP_out_of_sample_ND)
    
    precisions_ND.append(precision)

accuracies_ND = pd.DataFrame(accuracies_ND, columns = ["Train", "Test"], index = [method for method, x in methods.items()])